- 말뭉치: 자연어처리 연구나 애플리케이션을 염두에 두고 수집된 텍스트 데이터
- 통계기반기법의 목표: 사람의 지식으로 가득찬 말뭉치에서 자동으로, 그리고 효율적으로 그 핵심을 추출하는 것

## 2.3.1 파이썬으로 말뭉치 전처리하기
전처리란, 텍스트 데이터를 단어로 분할하고 그 분할된 단어들을 단어ID 목록으로 변환하는 일.

In [1]:
text = 'You say goodbye and I say hello.'

text = text.lower()
text = text.replace('.',' .')

text

'you say goodbye and i say hello .'

In [4]:
words = text.split(' ')           # 공백을 기준으로 분할
words

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello', '.']

In [6]:
# 딕셔너리를 이용하여 단어 ID와 단어를 짝지어주는 대응표 작성

word_to_id = {}
id_to_word= {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [7]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

In [8]:
word_to_id

{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}

In [9]:
id_to_word[1]

'say'

In [10]:
word_to_id['hello']

5

In [11]:
# '단어목록'을 'ID단어목록'으로 변경

import numpy as np

corpus = [word_to_id[w] for w in words]
corpus = np.array(corpus)
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [12]:
# preprocess()함수 구현

def preprocess(text):
    text = text.lower()
    text = text.replace('.',' .')
    words = text.split(' ')
   
    word_to_id = {}
    id_to_word= {}

    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
    
    corpus = np.array([word_to_id[w] for w in words])
    
    return corpus, word_to_id, id_to_word

In [13]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

## 2.3.2 단어의 분산 표현
- 분산표현 : 단어의 의미를 정확하게 파악할 수 있는 벡터 표현
- 단어의 분산 표현은 단어를 고정 길의 밀집벡터(dense vector)로 표현
- 밀집벡터는 대부분의 원소가 0이 아닌 실수인 벡터

## 2.3.3 분포 가설
- 단어의 의미는 주변 단어에 의해 형성된다. >> 분포 가설
- 단어 자체에는 의미가 없고, 그 단어가 사용된 '맥락'이 의미를 형성한다.

## 2.3.4 동시발생 행렬


In [15]:
import sys
sys.path.append('..')
import numpy as np
from common.util import preprocess

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

print(corpus)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [18]:
# 동시발생 행렬 파이썬 구현

C = np.array([
    [0,1,0,0,0,0,0],
    [1,0,1,0,1,1,0],
    [0,1,0,1,0,0,0],
    [0,0,1,0,1,0,0],
    [0,1,0,1,0,0,0],
    [0,1,0,0,0,0,1],
    [0,0,0,0,0,1,0],
], dtype=np.int32)

In [19]:
print(C[0])  # ID가 0 인 단어의 벡터 표현

[0 1 0 0 0 0 0]


In [20]:
print(C[4])  # ID가 4 인 단어의 벡터 표현

[0 1 0 1 0 0 0]


In [21]:
print(C[word_to_id['goodbye']])  # 'goodbye'의 벡터 표현

[0 1 0 1 0 0 0]


In [22]:
# 말뭉치로부터 동시발생 행렬을 만들어주는 함수

def create_co_matrix(corpus, vocab_size, window_size=1):     #단어ID의리스트, 어휘수, 윈도우크기
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size+1):
            left_idx = idx -i
            right_idx = idx +i
            
            if left_idx >= 0:
                lift_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1
                
            if right_idx > corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
                
    return co_matrix

- co_matrix를 0으로 채워진 2차원 배열로 초기화.
- 말뭉치의 모든 단어 각각에 대하여 윈도우에 포함된 주변 단어를 세어나감.
- 말뭉치의 왼쪽 끝과 오른쪽 끝 경계를 벗어나지 않는지 확인.

## 2.3.5 벡터 간 유사도
- 단어 벡터의 유사도를 나타낼 때는 코사인 유사도를 자주 이용.
- 분자에는 벡터의 내적, 분모에는 각 벡터의 norm. 여기서는 l2norm (벡터의 각 원소를 제곱해 더한후 다시 제곱근)
- 코사인 유사도의 핵심은 벡터를 정규화하고 내적을 구하는 것.
- 두 벡터가 가리키는 방향이 얼마나 비슷한가. 두 벡터의 방향이 완전히 같다면 코사인 유사도 1, 완전히 반대라면 -1

In [23]:
def cos_similarity(x, y, eps=1e-8):
    '''코사인 유사도 산출
    :param x: 벡터
    :param y: 벡터
    :param eps: '0으로 나누기'를 방지하기 위한 작은 값, eps = 엡실론의 약어
    :return:
    '''
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps)      # x의 정규화
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)      # y의 정규화 
    return np.dot(nx, ny)

In [24]:
# you와 i의 유사도를 구하는 코드

import sys
sys.path.append('..')
from common.util import preprocess, create_co_matrix, cos_similarity


text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id['you']]  # "you"의 단어 벡터
c1 = C[word_to_id['i']]    # "i"의 단어 벡터
print(cos_similarity(c0, c1))

0.7071067691154799


- 코사인 유사도 값은 -1 에서 1 사이이므로, 이 값은 비교적 높다(유사성이 크다)라고 할 수 있음.

## 2.3.6 유사단어 랭킹 표시
어떤 단어가 검색어로 주어지면, 그 검색어와 비슷한 단어를 유사도 순으로 출력하는 함수

In [25]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    '''유사 단어 검색
    :param query: 쿼리(텍스트)
    :param word_to_id: 단어에서 단어 ID로 변환하는 딕셔너리
    :param id_to_word: 단어 ID에서 단어로 변환하는 딕셔너리
    :param word_matrix: 단어 벡터를 정리한 행렬. 각 행에 해당 단어 벡터가 저장되어 있다고 가정한다.
    :param top: 상위 몇 개까지 출력할 지 지정
    '''
    # 검색어를 꺼낸다
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' % query)
        return

    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    # 코사인 유사도 계산
    vocab_size = len(id_to_word)

    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    # 코사인 유사도를 기준으로 내림차순으로 출력
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return

- 검색어의 단어 벡터를 꺼낸다.
- 검색어의 단어 벡터와 다른 모든 단어 벡터와의 코사인 유사도를 각각 구한다.
- 계산한 코사인 유사도 결과를 기준으로 값이 높은 순서대로 출력한다.

In [26]:
# you를 검색어로, 유사한 단어들 출력

import sys
sys.path.append('..')
from common.util import preprocess, create_co_matrix, most_similar


text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar('you', word_to_id, id_to_word, C, top=5)


[query] you
 goodbye: 0.7071067691154799
 i: 0.7071067691154799
 hello: 0.7071067691154799
 say: 0.0
 and: 0.0
